In [1]:
import findspark
findspark.init()

In [2]:
import os
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime
import re
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [3]:
conf = SparkConf().setMaster("local[4]").setAppName("transport")
sc = SparkContext(conf=conf)

sqlContext = SQLContext(sc)

In [4]:
def build_filename(raw_name):
    day, from_day, to_day = re.findall(r'\d+', raw_name)
    return "day=%s/from=%s/to=%s" % (day, from_day, to_day)

In [5]:
def extract_files(compressed_name, stream):
    """
    Extract .csv files from a .zip file and load in different DataFrames.
    """
    with BytesIO(stream) as tf:
        tf.seek(0)        
        # Read the file as a zipfile and process the members
        with ZipFile(tf, mode='r') as zipf:
            return [(build_filename(compressed_name + file_name), zipf.read(file_name)) for file_name in zipf.namelist()]

In [6]:
# datetime.strptime(row[0], "%d/%m/%Y %H:%M:%S")
def prepare_csv(file_name, table):
    return list(map(lambda row: [file_name, row[0], int(row[1]), 
                                 row[2], int(row[3]), row[4], 
                                 row[5]], table)) 

In [7]:
path = 'datasets/test-folder/*.zip'
rdd = sc.binaryFiles(path).flatMap(lambda a: extract_files(a[0], a[1]))
print(rdd.toDebugString().decode())

(2) PythonRDD[1] at RDD at PythonRDD.scala:53 []
 |  datasets/test-folder/*.zip BinaryFileRDD[0] at binaryFiles at NativeMethodAccessorImpl.java:0 []


In [8]:
rdd.getNumPartitions()

2

In [9]:
# Decode bytes and convert it in a list of strings
rdd = rdd.mapValues(lambda file: BytesIO(file).read().decode('cp1252').split('\n'))

In [10]:
# Convert string row to list row
rdd = rdd.mapValues(lambda a: list(map(lambda row: row.split(";")[:-1], a)))

In [11]:
# Drop header and last (and empty) row
rdd = rdd.mapValues(lambda table: table[1:-1])

In [12]:
# Change type of columns
rdd = rdd.map(lambda a: prepare_csv(a[0], a[1]))

In [13]:
rdd = rdd.flatMap(lambda a: prepare_csv(a[0], a[1]))

In [14]:
header = ['FILE_NAME',
            'FECHAHORATRX',
            'CODIGOENTIDAD',
            'NOMBREENTIDAD',
            'CODIGOSITIO',
            'NOMBRESITIO',
            'NROTARJETA']
header = list(map(lambda a: a.lower(), header))
df = rdd.toDF(header)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-13-84579c49bbb1>", li

In [62]:
df.show()

+--------------------+-------------------+-------------+-------------+-----------+-----------+--------------------+
|           file_name|       fechahoratrx|codigoentidad|nombreentidad|codigositio|nombresitio|          nrotarjeta|
+--------------------+-------------------+-------------+-------------+-----------+-----------+--------------------+
|day=20190311/from...|01/03/2019 01:20:44|           16|    U3 - Vule|       2308|    BJFZ-75|37d3a2cd6ba280f3a...|
|day=20190311/from...|01/03/2019 01:20:47|           16|    U3 - Vule|       2308|    BJFZ-75|7464f0fb6fe00e347...|
|day=20190311/from...|01/03/2019 01:20:50|           16|    U3 - Vule|       2308|    BJFZ-75|3ce1dbdd2d904a953...|
|day=20190311/from...|01/03/2019 01:20:53|           16|    U3 - Vule|       2308|    BJFZ-75|dd9730ec384f81487...|
|day=20190311/from...|01/03/2019 01:21:00|           16|    U3 - Vule|       2308|    BJFZ-75|7f4ded632ab214988...|
|day=20190311/from...|01/03/2019 01:21:06|           16|    U3 - Vule|  

In [69]:
day = 'day=20190311/from=20190307/to=20190309'
df.select(df.columns[1:]).where(df.file_name == day).count()

99

In [74]:
days = df.select('file_name').distinct().collect()
days

[Row(file_name='day=20190311/from=20190307/to=20190309'),
 Row(file_name='day=20180818/from=20180812/to=20180814')]

In [80]:
days = [i.file_name for i in df.select('file_name').distinct().collect()]
days

['day=20190311/from=20190307/to=20190309',
 'day=20180818/from=20180812/to=20180814']

In [84]:
for directory in days:
    if not os.path.exists(directory):
        os.makedirs(directory)
    df_day = df.select(df.columns[1:]).where(df.file_name == day)
    df_day.write.parquet(directory + "/data.parquet", compression="gzip")